In [1]:
import os
import pandas as pd

SCORE_DIR = "data/scores"

In [2]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [3]:
from bs4 import BeautifulSoup

def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [4]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [5]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [6]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [7]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

/tmp/ipykernel_3041/2138878849.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future

100 / 5224


/tmp/ipykernel_3041/2138878849.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future

200 / 5224


/tmp/ipykernel_3041/2138878849.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future

...

/tmp/ipykernel_3041/2138878849.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future

700 / 5224


/tmp/ipykernel_3041/2138878849.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future

800 / 5224


/tmp/ipykernel_3041/2138878849.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_3041/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future

In [ ]:
games_df = pd.concat(games, ignore_index=True)

In [ ]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,30.0,73.0,0.411,8.0,25.0,0.320,22.0,29.0,...,25.0,35.1,200.0,106.0,TOR,102,1,2022,2021-12-05,False
1,240.0,240.0,38.0,85.0,0.447,8.0,23.0,0.348,18.0,24.0,...,50.0,38.3,200.0,121.0,WAS,90,0,2022,2021-12-05,True
2,240.0,240.0,42.0,75.0,0.560,11.0,29.0,0.379,16.0,19.0,...,29.1,34.0,235.0,118.0,DEN,117,1,2022,2022-01-23,False
3,240.0,240.0,44.0,85.0,0.518,12.0,35.0,0.343,17.0,23.0,...,37.5,36.3,155.0,126.0,DET,111,0,2022,2022-01-23,True
4,240.0,240.0,41.0,93.0,0.441,13.0,36.0,0.361,21.0,28.0,...,50.0,27.0,154.0,120.0,MIA,108,1,2023,2022-10-19,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7865,240.0,240.0,44.0,85.0,0.518,16.0,30.0,0.533,13.0,19.0,...,19.2,30.2,300.0,130.0,POR,109,0,2022,2021-11-09,True
7866,240.0,240.0,46.0,76.0,0.605,13.0,28.0,0.464,22.0,27.0,...,24.1,40.9,179.0,142.0,POR,116,1,2024,2024-01-14,True
7867,240.0,240.0,40.0,89.0,0.449,20.0,42.0,0.476,16.0,22.0,...,29.6,29.8,199.0,130.0,PHO,127,0,2024,2024-01-14,False
7868,240.0,240.0,37.0,93.0,0.398,17.0,52.0,0.327,0.0,0.0,...,46.7,33.5,186.0,114.0,MIL,104,1,2024,2024-04-09,False


In [ ]:
games_df.to_csv("nba_games.csv")